In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
BASE = "/kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup"
GENRES_DIR = f"{BASE}/genres_stems"
MASHUPS_DIR = f"{BASE}/mashups"
ESC_DIR = f"{BASE}/ESC-50-master/audio"

In [3]:
import os

genres = sorted(os.listdir(GENRES_DIR))

counts = {}

for g in genres:
    gpath = os.path.join(GENRES_DIR, g)
    songs = os.listdir(gpath)
    counts[g] = len(songs)

counts


{'blues': 100,
 'classical': 100,
 'country': 100,
 'disco': 100,
 'hiphop': 100,
 'jazz': 100,
 'metal': 100,
 'pop': 100,
 'reggae': 100,
 'rock': 100}

In [4]:
from collections import Counter

stem_counter = Counter()

for g in genres:
    for song in os.listdir(os.path.join(GENRES_DIR, g)):
        sp = os.path.join(GENRES_DIR, g, song)
        if os.path.isdir(sp):
            for f in os.listdir(sp):
                stem_counter[f] += 1

stem_counter


Counter({'drums.wav': 1000,
         'vocals.wav': 1000,
         'bass.wav': 1000,
         'other.wav': 1000})

In [5]:
len(os.listdir(MASHUPS_DIR))


3020

In [6]:
import librosa
import numpy as np
from tqdm import tqdm

durations = []

files = os.listdir(MASHUPS_DIR)[:200]   # sample only (fast EDA)

for f in tqdm(files):
    y, sr = librosa.load(os.path.join(MASHUPS_DIR, f), sr=None)
    durations.append(len(y)/sr)

np.min(durations), np.max(durations), np.mean(durations)


100%|██████████| 200/200 [00:22<00:00,  9.00it/s]


(np.float64(8.70748299319728),
 np.float64(30.370975056689343),
 np.float64(28.57347664399093))

In [7]:
import pandas as pd

esc_meta = pd.read_csv(f"{BASE}/ESC-50-master/meta/esc50.csv")
esc_meta.head()


,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


In [8]:
esc_meta["category"].value_counts()


category
dog                 40
chirping_birds      40
vacuum_cleaner      40
thunderstorm        40
door_wood_knock     40
can_opening         40
crow                40
clapping            40
fireworks           40
chainsaw            40
airplane            40
mouse_click         40
pouring_water       40
train               40
sheep               40
water_drops         40
church_bells        40
clock_alarm         40
keyboard_typing     40
wind                40
footsteps           40
frog                40
cow                 40
brushing_teeth      40
car_horn            40
crackling_fire      40
helicopter          40
drinking_sipping    40
rain                40
insects             40
laughing            40
hen                 40
engine              40
breathing           40
crying_baby         40
hand_saw            40
coughing            40
glass_breaking      40
snoring             40
toilet_flush        40
pig                 40
washing_machine     40
clock_tick          40
sn

In [9]:
noise_files = os.listdir(ESC_DIR)[:200]

noise_durs = []

for f in noise_files:
    y, sr = librosa.load(os.path.join(ESC_DIR, f), sr=None)
    noise_durs.append(len(y)/sr)

np.min(noise_durs), np.max(noise_durs)


(np.float64(5.0), np.float64(5.0))

In [10]:
import os
import librosa
import numpy as np

energies = {}

example = os.path.join(GENRES_DIR, genres[0], os.listdir(os.path.join(GENRES_DIR, genres[0]))[0])

for s in ["drums.wav","bass.wav","vocals.wav","others.wav"]:
    path = os.path.join(example, s)

    try:
        y, sr = librosa.load(path, sr=None, mono=True)
        energies[s] = float(np.mean(y**2))
    except Exception as e:
        print("Failed to load:", path)
        print(type(e).__name__)
        energies[s] = None

energies


Failed to load: /kaggle/input/jan-2026-dl-gen-ai-project/messy_mashup/genres_stems/blues/blues.00095/others.wav
FileNotFoundError


/tmp/ipykernel_17/3653445495.py:13: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(path, sr=None, mono=True)
/usr/local/lib/python3.12/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


{'drums.wav': 2.8061806006007828e-05,
 'bass.wav': 0.015466556884348392,
 'vocals.wav': 0.015071025118231773,
 'others.wav': None}

In [11]:
import pandas as pd

test_df = pd.read_csv(f"{BASE}/test.csv")
test_df.head()


,id,filename
0,1,mashups/song0001.wav
1,2,mashups/song0002.wav
2,3,mashups/song0003.wav
3,4,mashups/song0004.wav
4,5,mashups/song0005.wav


In [12]:
import random

LABELS = [
 "blues","classical","country","disco","hiphop",
 "jazz","metal","pop","reggae","rock"
]

preds = []

for _ in range(len(test_df)):
    preds.append(random.choice(LABELS))




In [13]:
def rule_based(filename):
    for g in LABELS:
        if g in filename.lower():
            return g
    return random.choice(LABELS)

preds = []

for _, row in test_df.iterrows():
    preds.append(rule_based(row["filename"]))

submission = pd.DataFrame({
    "id": test_df["id"],
    "genre": preds
})

submission.to_csv("submission.csv", index=False)
